In [1]:
from db_connection import DB_Connection

In [2]:
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
import re, json
from wordcloud import WordCloud

In [3]:
db = DB_Connection()
database = db.query_all()

Connection succesful.


In [4]:
df = pd.DataFrame(database, columns=['id','userid','username','text','date',
                            'link','media1','media2','media3', 'media4'])
df_clean = df[:]
df_clean = df_clean[df_clean.text!='']

In [55]:
# Cleaning the tweets
stop_words = json.load(open('./others/stop_words.json', encoding='utf8'))
def preprocess(tweet):
    # remove next lines
    tweet = re.sub('\n', ' ', tweet)
    # remove links
    tweet = re.sub('http\S+\s*', '', tweet)
    # remove between parentesis
    tweet = re.sub('\[.*\]','', tweet)
    # remove mentions
    tweet = re.sub("@\w+", "", tweet)
    # remove hashtags
    tweet = re.sub("#\w+", "", tweet)
    # alphanumeric and hashtags
    tweet = re.sub("[^a-zA-Z0-9ñáéíóúÁÉÍÓÚüÜ]", " ", tweet)
    # remove multiple spaces
    tweet = re.sub("\s+", " ", tweet)
    tweet = re.sub('^\s+', '', tweet)
    # lower first character
    try: tweet = tweet[0].lower() + tweet[1:]
    except: pass
    # remove stop words
    tweet = ' '.join([word for word in tweet.split(' ') 
                if not word.lower() in stop_words])

    return tweet

In [ ]:
all_words = ' '.join([preprocess(t) for t in df.text])

In [61]:
# df_processed = pd.DataFrame(columns=['id','text','clean','place','coords'])
df_processed = df_clean.loc[:, ['id', 'text']]
df_processed['clean'] = df_processed.text
df_processed['clean'] = df_processed.apply(lambda row: preprocess(row.clean), axis=1)
# for row in df:
#     index = len(df_processed)
#     print(row)
#     df_processed.iloc[index, 'id'] = row[0]
#     df_processed.iloc[index, 'text'] = row[3]
#     clean_twt = preprocess(row[3])
#     df_processed.iloc[index, 'clean'] = clean_twt
#     df_processed.iloc[index, 'place'] = get_geoloc(clean_twt)

df_processed

,id,text,clean
0,1349845441026912263,Reportan que se registró una balacera en Santa...,reportan registró balacera Santa María avenida...
1,1349844326575177732,[LOÚLTIMO] Un muerto deja balacera en Santa Ma...,muerto deja balacera Santa Maria Avenida Ricar...
2,1349833820850290690,Vistas del tráfico en el puente de Las América...,vistas tráfico puente Américas dirección Panam...
3,1349832612756545536,A 5km/h se desplazan los autos en la Vía Inter...,5km h desplazan autos Vía Interamericana Howar...
4,1349832038250131458,Reportan accidente de tránsito en la avenida L...,reportan accidente tránsito avenida Américas C...
...,...,...,...
175811,423098320366477312,"#Panama ""\n@CriticaPa\n: NACIONAL: Quijano: AC...",nACIONAL Quijano ACP recibido constancia escri...
175812,423098157291933696,El Canal de Panamá sirve a 144 rutas marítimas...,Canal Panamá sirve 144 rutas marítimas conecta...
175813,423094695871254528,"""\n@LosSantos7\n: Gran victoria la de anoche v...",victoria anoche vs Metro 1 5 Olmedo Solé
175814,423088950601732097,"""\n@tvnnoticias\n: ¿Está de acuerdo con sancio...",acuerdo sanciones billeteros vendan chances bi...


### WordCloud

In [ ]:
cloud = WordCloud(width=1500, height=900).generate(all_words)
plt.figure(figsize=(15,9))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [ ]:
# Find places
places = ' '.join(re.findall('([A-Z]\S+)', all_words))
cloud2 = WordCloud(width=1500, height=900).generate(places)
plt.figure(figsize=(15,9))
plt.imshow(cloud2)
plt.axis('off')
plt.show()

### Getting geolocation

In [25]:
from geopy.geocoders import Nominatim
gp = Nominatim(user_agent='data-clean')

In [26]:
def get_place(tweet):
    places = ' '.join(re.findall('([A-Z]\S+)', tweet))
    # geo = gp.geocode('places')
    return places

In [27]:
def get_geoloc(place):
    geo = gp.geocode(place)
    return ((geo.latitude, geo.longitude) if geo!=None else (None,None))

In [28]:
print(get_geoloc('Capira'))

(8.81612315, -80.04228931584623)


In [62]:
df_processed['place'] = df_processed.apply(lambda row: get_place(row.clean), axis=1)
# df_processed['lat'] = df_processed.apply(lambda row: get_geoloc(row.place)[0], axis=1)
# df_processed['long'] = df_processed.apply(lambda row: get_geoloc(row.place)[1], axis=1)
df_processed['lat'] = ''
df_processed['long'] = ''
df_processed['isIncident'] = ''
df_processed['isAccident'] = ''
df_processed['isObstacle'] = ''
df_processed['isDanger'] = ''

In [63]:
df2 = df_processed[:]

In [49]:
df2['isIncident'] = df2.apply(lambda row: 1 if any(x in row.clean for x in ['accidente','accidentaron','choque','chocaron','colisión','colisionaron','colisionó','vuelco','volcó','trabajando','trabajo','mantenimiento','cierre','cerraron','daño','dañó','dañado','árbol','incendio','incendiando','incendiado','fuego','inundado','inundación','inundó','tranque','trancado','embotellamiento','huelga','motín','protesta','protestando','obstáculo','obstaculizando','parado','paro']) else 0, axis=1)

In [86]:
df2['isAccident'] = np.where(df2.clean.str.contains('accidente|accidentaron|accidentó|choque|chocó|chocaron|choca|chocar|colisión|colisionaron|colisionó|colisiona|colisionado|vuelco|volcó|vuelca|atropello|atropellado'), 1, 0)

df2['isObstacle'] = np.where(df2.clean.str.contains('tranque|trancado|embotellamiento|desplazan autos|vistas tráfico|movimiento vehicular|huelga|motín|protesta|protestando|trabajos vía|trabajos|trabajos ruta|cierre|cerraron|cerrado|cierran|cierra|daño|dañó|detuvieron|detenido|detenida|obstáculo|obstaculizando|parado|paro|paño cerrado|tráfico paralizado|tráfico detenido|tráfico afectado|tráfico pesado|tráfico lento|desvío|área acordonada'), 1, 0)

df2['isDanger'] = np.where(df2.clean.str.contains('incendio|incendia|incendiando|incendiado|incendiaron|inundado|inundación|inundó'), 1, 0)

df2['isIncident'] = np.where(df2[['isAccident','isObstacle','isDanger']].sum(axis=1)>0, 1, 0)

In [87]:
df2

,id,text,clean,place,lat,long,isIncident,isAccident,isObstacle,isDanger
0,1349845441026912263,Reportan que se registró una balacera en Santa...,reportan registró balacera Santa María avenida...,Santa María Ricardo Alfaro Unidades Policía,,,0,0,0,0
1,1349844326575177732,[LOÚLTIMO] Un muerto deja balacera en Santa Ma...,muerto deja balacera Santa Maria Avenida Ricar...,Santa Maria Avenida Ricardo Alfaro Unidades Po...,,,0,0,0,0
2,1349833820850290690,Vistas del tráfico en el puente de Las América...,vistas tráfico puente Américas dirección Panam...,Américas Panamá Oeste,,,1,0,1,0
3,1349832612756545536,A 5km/h se desplazan los autos en la Vía Inter...,5km h desplazan autos Vía Interamericana Howar...,Vía Interamericana Howard Panamá Oeste,,,1,0,1,0
4,1349832038250131458,Reportan accidente de tránsito en la avenida L...,reportan accidente tránsito avenida Américas C...,Américas Chorrera Panamá Oeste,,,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
175811,423098320366477312,"#Panama ""\n@CriticaPa\n: NACIONAL: Quijano: AC...",nACIONAL Quijano ACP recibido constancia escri...,ACIONAL Quijano ACP,,,0,0,0,0
175812,423098157291933696,El Canal de Panamá sirve a 144 rutas marítimas...,Canal Panamá sirve 144 rutas marítimas conecta...,Canal Panamá,,,0,0,0,0
175813,423094695871254528,"""\n@LosSantos7\n: Gran victoria la de anoche v...",victoria anoche vs Metro 1 5 Olmedo Solé,Metro Olmedo Solé,,,0,0,0,0
175814,423088950601732097,"""\n@tvnnoticias\n: ¿Está de acuerdo con sancio...",acuerdo sanciones billeteros vendan chances bi...,Comente,,,0,0,0,0


In [88]:
df2.to_csv('data.csv')

In [66]:
for row in df_processed.head(5).text:
    print(row)

Reportan que se registró una balacera en Santa María, cerca a la avenida Ricardo J. Alfaro. Según información preliminar una persona perdió la vida. Unidades de la Policía se mantienen el área. 
@TraficoCPanama
 #TReporta
[LOÚLTIMO] Un muerto deja balacera en Santa Maria, cerca a la Avenida Ricardo J Alfaro. Unidades de la Policía Nacional mantiene el área acordonada.
Vistas del tráfico en el puente de Las Américas, en dirección a Panamá Oeste.
A 5km/h se desplazan los autos en la Vía Interamericana, cerca Howard en dirección a Panamá Oeste.
Reportan accidente de tránsito en la avenida Las Américas, La Chorrera, provincia de Panamá Oeste. Mucha precaución por el área. 
@TraficoCPanama
 #TReporta


## Some preprocessing tests

In [ ]:
str1 = '    Hola desde la Capital de Panama @traficopanama #tranque'
str2 = preprocess(str1)
str2 = 

In [ ]:
# str1 = re.sub('^\s*', '', str1)
# str3 = ' '.join([word for word in str1.split(' ') if word.contains('[A-Z]')])
# str3
re.findall('([A-Z]\S+)', str1)

In [ ]:
cad = '[Vídeo] [LOULTIMO] hola desde aqui'
cad = re.sub('\[.*\]','', cad)
cad = re.sub('^\s+', '', cad)
cad

In [ ]:
t = re.sub('|'.join(stop_words), '', 'hola en la ola yo estoy aquí')
t = re.sub('\s+', ' ', t)
t

In [ ]:
re.sub('asdf', '0', 'asdfasdf')